In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.pipeline import make_pipeline



#algoritmi di classificazione
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#metrica di confronto
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('datasets/recipeData.csv',encoding='latin-1')
df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0


# Esplorazione dei dati

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73861 entries, 0 to 73860
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   BeerID         73861 non-null  int64  
 1   Name           73860 non-null  object 
 2   URL            73861 non-null  object 
 3   Style          73265 non-null  object 
 4   StyleID        73861 non-null  int64  
 5   Size(L)        73861 non-null  float64
 6   OG             73861 non-null  float64
 7   FG             73861 non-null  float64
 8   ABV            73861 non-null  float64
 9   IBU            73861 non-null  float64
 10  Color          73861 non-null  float64
 11  BoilSize       73861 non-null  float64
 12  BoilTime       73861 non-null  int64  
 13  BoilGravity    70871 non-null  float64
 14  Efficiency     73861 non-null  float64
 15  MashThickness  43997 non-null  float64
 16  SugarScale     73861 non-null  object 
 17  BrewMethod     73861 non-null  object 
 18  PitchR

In [4]:
df.describe()

,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,MashThickness,PitchRate,PrimaryTemp,UserId
count,73861.000000,73861.000000,73861.000000,73861.000000,73861.000000,73861.000000,73861.000000,73861.000000,73861.000000,73861.000000,70871.000000,73861.000000,43997.000000,34609.000000,51199.000000,23371.000000
mean,36931.000000,60.179432,43.929775,1.406266,1.075865,6.136865,44.276186,13.404989,49.724919,65.074870,1.353955,66.354881,2.127235,0.750468,19.175641,43078.069188
std,21321.978453,56.811462,180.373492,2.196908,0.432524,1.883510,42.945508,11.944511,193.246427,15.024228,1.930989,14.091686,1.682347,0.394262,4.219676,27734.252556
min,1.000000,1.000000,1.000000,1.000000,-0.003000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-17.780000,49.000000
25%,18466.000000,10.000000,18.930000,1.051000,1.011000,5.080000,23.370000,5.170000,20.820000,60.000000,1.040000,65.000000,1.500000,0.350000,18.000000,20984.000000
50%,36931.000000,35.000000,20.820000,1.058000,1.013000,5.790000,35.770000,8.440000,27.440000,60.000000,1.047000,70.000000,1.500000,0.750000,20.000000,42897.000000
75%,55396.000000,111.000000,23.660000,1.069000,1.017000,6.830000,56.380000,16.790000,30.000000,60.000000,1.060000,75.000000,3.000000,1.000000,20.000000,57841.000000
max,73861.000000,176.000000,9200.000000,34.034500,23.424600,54.720000,3409.300000,186.000000,9700.000000,240.000000,52.600000,100.000000,100.000000,2.000000,114.000000,134362.000000


In [5]:
print(df.isnull().sum().sort_values(ascending=False))
print(df.isnull().sum().sort_values(ascending=False).index[:4])

PrimingAmount    69087
PrimingMethod    67095
UserId           50490
PitchRate        39252
MashThickness    29864
PrimaryTemp      22662
BoilGravity       2990
Style              596
Name                 1
BrewMethod           0
SugarScale           0
Efficiency           0
BeerID               0
BoilTime             0
Color                0
IBU                  0
ABV                  0
FG                   0
OG                   0
Size(L)              0
StyleID              0
URL                  0
BoilSize             0
dtype: int64
Index(['PrimingAmount', 'PrimingMethod', 'UserId', 'PitchRate'], dtype='object')


# Data Pre-processing

## Gestione dei valori vuoti

In [6]:
#eliminazione di colonne molto sparsi
df=df.drop(columns=df.isnull().sum().sort_values(ascending=False).index[:6])
df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,SugarScale,BrewMethod
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,Specific Gravity,All Grain
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,Specific Gravity,All Grain
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,Specific Gravity,extract
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,Specific Gravity,All Grain
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,Specific Gravity,All Grain


In [7]:
#gestione della colonna BoilGravity
df['BoilGravity'].fillna(df['BoilGravity'].mean(),inplace=True)
df['BoilGravity'].isnull().sum()

0

In [8]:
#gestione della colonna stile
df = df.dropna(subset=['Style'])
df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,SugarScale,BrewMethod
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038000,70.0,Specific Gravity,All Grain
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070000,70.0,Specific Gravity,All Grain
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,1.353955,70.0,Specific Gravity,extract
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,1.353955,70.0,Specific Gravity,All Grain
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050000,72.0,Specific Gravity,All Grain


In [9]:
df.isnull().sum().sort_values(ascending=False)

Name           1
BeerID         0
IBU            0
SugarScale     0
Efficiency     0
BoilGravity    0
BoilTime       0
BoilSize       0
Color          0
ABV            0
FG             0
OG             0
Size(L)        0
StyleID        0
Style          0
URL            0
BrewMethod     0
dtype: int64

## Selezione delle variabili

In [10]:
#eliminazione delle variabili considerati inutili secondo l'esperto
df = df.drop(columns=['BeerID','Name','URL','Style'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73265 entries, 0 to 73860
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   StyleID      73265 non-null  int64  
 1   Size(L)      73265 non-null  float64
 2   OG           73265 non-null  float64
 3   FG           73265 non-null  float64
 4   ABV          73265 non-null  float64
 5   IBU          73265 non-null  float64
 6   Color        73265 non-null  float64
 7   BoilSize     73265 non-null  float64
 8   BoilTime     73265 non-null  int64  
 9   BoilGravity  73265 non-null  float64
 10  Efficiency   73265 non-null  float64
 11  SugarScale   73265 non-null  object 
 12  BrewMethod   73265 non-null  object 
dtypes: float64(9), int64(2), object(2)
memory usage: 7.8+ MB


In [11]:
#gestione dei dati categorici
le = LabelEncoder()
df['SugarScale'] = le.fit_transform(df['SugarScale'].values)
df['BrewMethod'] = le.fit_transform(df['BrewMethod'].values)
df[['SugarScale','BrewMethod']].head()

,SugarScale,BrewMethod
0,1,0
1,1,0
2,1,3
3,1,0
4,1,0


# Implementazione del modello

In [17]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold

pipe_lr = make_pipeline(StandardScaler(),LDA(n_components=2),LogisticRegression(random_state=0))
pipe_tree = make_pipeline(DecisionTreeClassifier(random_state=0,max_features=5))
pipe_knn = make_pipeline(StandardScaler(),LDA(n_components=2),KNeighborsClassifier(n_neighbors=10,weights='distance'))

ksplits = StratifiedKFold(n_splits=10)

In [21]:
import numpy as np
X = df.drop(columns=['StyleID']).values
y = df['StyleID'].values

#Discriminante Logistico
scores_lr = []
for train,test in ksplits.split(X,y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    pipe_lr.fit(X_train,y_train)
    y_pred = pipe_lr.predict(X_test)
    scores_lr.append(accuracy_score(y_test,y_pred))

#KNN
scores_knn = []
for train,test in ksplits.split(X,y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    pipe_knn.fit(X_train,y_train)
    y_pred = pipe_knn.predict(X_test)
    scores_knn.append(accuracy_score(y_test,y_pred))

#Albero decisionale
scores_tree = []
for train,test in ksplits.split(X,y):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
    pipe_tree.fit(X_train,y_train)
    y_pred = pipe_tree.predict(X_test)
    scores_tree.append(accuracy_score(y_test,y_pred))

In [23]:
print(f'Albero Decisionale:{np.mean(scores_tree):.3f} +/- {np.std(scores_tree):.3f}')
print(f'KNN:{np.mean(scores_knn):.3f} +/- {np.std(scores_knn):.3f}')
print(f'Discriminante Logistico:{np.mean(scores_lr):.3f} +/- {np.std(scores_lr):.3f}')

Albero Decisionale:0.223 +/- 0.004
KNN:0.221 +/- 0.004
Discriminante Logistico:0.249 +/- 0.004
